In [1]:
# Importing necessary libraries and connecting to civis platform
import pandas as pd
import numpy as np
from datetime import datetime
import civis
client = civis.APIClient()
db = "UN High Commissioner for Refugees"

In [2]:
# Query to select the necessary variables
contacts_demographics = """SELECT a.source_id,
                                  c.accountid,
                                  c.firstname,
                                  c.middlename,
                                  c.lastname,
                                  c.primary_address as street,
                                  c.secondary_address as street2,
                                  c.city,
                                  c.state,
                                  c.zip,
                                  c.county,
                                  b.coalesced_noncommercial_age,
                                  b.race5way_noncommercial,
                                  b.subethnicity_noncommercial,
                                  b.gender_noncommercial,
                                  b.marriage_noncommercial,
                                  b.parent_noncommercial,
                                  b.religion_noncommercial,
                                  b.household_income_bucket,
                                  b.household_net_worth_bucket,
                                  b.vf_reg_party,
                                  b.vb_dob
                                  
                             FROM staging.sf_donors_vb_id a
                                  LEFT JOIN ts.basic_noncommercial_client b 
                                      ON a.matched_id = b.voterbase_id
                                  LEFT JOIN staging.sf_donors_for_person_matching c 
                                      ON c.id = a.source_id
"""

In [3]:
dtype_dict = {'zip': str,  'vf_reg_party':str, 'race5way_noncommercial':str, 'subethnicity_noncommercial':str,
              'gender_noncommercial':str, 'marriage_noncommercial':str, 'parent_noncommercial':str, 'religion_noncommercial':str,
              'household_income_bucket':str, 'household_net_worth_bucket':str}          

In [4]:
# Reading the query from civis
contacts_demographics = civis.io.read_civis_sql(contacts_demographics, database = db, use_pandas=True, dtype = dtype_dict)

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
# Fuction to reorganize the variable party
def party(party):
    if party in ['Independent','Green','Other','Libertarian','Conservative','Working Fam']:
        return 'Others'
    if party in ['Unaffiliated','No Party']:
        return 'Unaffiliated'
    if party == 'Unknown':
        return 'UNKNOWN'
    if party is np.nan:
        return 'UNKNOWN'
    else:
        return party

In [7]:
def race(race):
    if race == 'AfAm':
        return 'African American'
    if race == 'Native':
        return 'Native American'
    if race is np.nan:
        return 'UNKNOWN'
    if race == 'Unknown':
        return 'UNKNOWN'
    else:
        return race

In [8]:
# Function to categorize age 
def age(age):
    if age < 18:
        return 'Less than 18'
    if (age>=18) & (age<=24):
        return '18-24'
    if (age>=25) & (age<=34):
        return '25-34'
    if (age>=35) & (age<=44):
        return '35-44'
    if (age>=45) & (age<=54):
        return '45-54'
    if (age>=55) & (age<=64):
        return '55-64'
    if (age>=65) & (age<=74):
        return '65-74'
    if (age>=75) & (age<=84):
        return '75-84'
    if age>=85:
        return 'Greater than 85'
    if age is np.nan:
        return 'UNKNOWN'
    else:
        return 'UNKNOWN'

In [9]:
def convert_date(date):
    return datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')

In [10]:
# Create the variable date of birth
contacts_demographics['vb_dob2'] = contacts_demographics['vb_dob'].fillna(18000101.0)
contacts_demographics['vb_dob2'] = contacts_demographics['vb_dob2'].astype(str)
contacts_demographics['vb_dob2'] = contacts_demographics['vb_dob2'].apply(lambda x: x.split('.')[0])
contacts_demographics['vb_dob2'] = contacts_demographics['vb_dob2'].apply(convert_date)
contacts_demographics['vb_dob2'] = np.where(contacts_demographics['vb_dob2']=='1800-01-01',np.nan,contacts_demographics['vb_dob2'])
contacts_demographics.drop('vb_dob',axis=1,inplace=True)

In [11]:
# Dictionary to map old names to new names
rename_dict = {'source_id':'contactid',
               'coalesced_noncommercial_age':'age',
               'race5way_noncommercial':'race',
               'subethnicity_noncommercial':'subethnicity',
               'gender_noncommercial':'gender',
               'marriage_noncommercial':'marital_status',
               'parent_noncommercial':'parent',
               'religion_noncommercial':'religion',
               'household_income_bucket':'income',
               'household_net_worth_bucket':'household_net_worth_bucket',
               'vf_reg_party':'party',
               'vb_dob2':'date_of_birth'}

In [12]:
# Renaming columns
contacts_demographics = contacts_demographics.rename(columns = rename_dict)

In [13]:
# Applying functions to make changes on the variables race and party, and create new variable age_bucket
contacts_demographics['race'] = contacts_demographics['race'].map(lambda x: race(x))
contacts_demographics['age_bucket'] = contacts_demographics['age'].map(lambda x: age(x))
contacts_demographics['party'] = contacts_demographics['party'].map(lambda x: party(x))

In [14]:
cols = ['gender','race','age_bucket','age','subethnicity','parent','income','party','marital_status',
          'religion','household_net_worth_bucket','date_of_birth']

# Replacing missing values with UNKNOWN
contacts_demographics[cols] = contacts_demographics[cols].fillna('UNKNOWN')

In [15]:
contacts_demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830480 entries, 0 to 1830479
Data columns (total 23 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   contactid                   object
 1   accountid                   object
 2   firstname                   object
 3   middlename                  object
 4   lastname                    object
 5   street                      object
 6   street2                     object
 7   city                        object
 8   state                       object
 9   zip                         object
 10  county                      object
 11  age                         object
 12  race                        object
 13  subethnicity                object
 14  gender                      object
 15  marital_status              object
 16  parent                      object
 17  religion                    object
 18  income                      object
 19  household_net_worth_bucket  object
 20  pa

In [16]:
contacts_demographics.head()

,contactid,accountid,firstname,middlename,lastname,street,street2,city,state,zip,county,age,race,subethnicity,gender,marital_status,parent,religion,income,household_net_worth_bucket,party,date_of_birth,age_bucket
0,0034100002UhFSOAA3,0014100001yoNlnAAE,James,NaN,Mitchell,4592 S TONGASS HWY,NaN,KETCHIKAN,AK,99901,KETCHIKAN GATEWAY,68.0,White,White,Male,Married,Non-Parent,Evangelical Protestant,$150k or more,$750k or more,Unaffiliated,1954-03-01,65-74
1,0031K00002hY68QQAS,0011K00002ApLzdQAF,Kathleen,NaN,Claiborne,2621 E WANAMINGO DR,NaN,WASILLA,AK,99654,MATANUSKA SUSITNA,73.0,White,White,Female,Unmarried,Non-Parent,Mainline Protestant,$100k-$149k,$250k-$749k,Unaffiliated,1949-01-21,65-74
2,00341000019f9ejAAA,0014100000xPVN1AAO,Gary,NaN,Pounds,7921 ASCOT ST,NaN,ANCHORAGE,AK,99502,ANCHORAGE,68.0,White,White,Male,Married,Non-Parent,Mainline Protestant,$150k or more,$750k or more,Democrat,1954-12-01,65-74
3,0031K00002q57zpQAA,0011K00002HXNHsQAP,Samantha,NaN,Scott,2443 SPURR LN,NaN,ANCHORAGE,AK,99517,ANCHORAGE,36.0,White,White,Female,Married,Parent,Mainline Protestant,$75k-$99k,$250k-$749k,Democrat,1986-04-09,35-44
4,0031K00003ANRnGQAX,0011K00002V9hkfQAB,Charles,NaN,Michael,2421 WELLINGTON CT,NaN,ANCHORAGE,AK,99517,ANCHORAGE,72.0,White,White,Male,Married,Non-Parent,Catholic,$150k or more,$750k or more,Unaffiliated,1950-07-10,65-74


##### Changing all California residents demographic to UNKNOWN

In [17]:
contacts_demographics.loc[contacts_demographics["state"] == "CA", "age"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "race"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "subethnicity"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "gender"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "marital_status"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "parent"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "religion"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "income"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "household_net_worth_bucket"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "party"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "date_of_birth"] = 'UNKNOWN'
contacts_demographics.loc[contacts_demographics["state"] == "CA", "age_bucket"] = 'UNKNOWN'

In [18]:
df = contacts_demographics[contacts_demographics["state"] == "CA"]
df.head()

,contactid,accountid,firstname,middlename,lastname,street,street2,city,state,zip,county,age,race,subethnicity,gender,marital_status,parent,religion,income,household_net_worth_bucket,party,date_of_birth,age_bucket
90,0031K00002rC9pYQAS,0011K00002IIa0BQAT,William,D,Cox,3051A SUNSET AVE,NaN,MARINA,CA,93933,MONTEREY,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
141,0031K00002ZGK4jQAH,0011K000023IjZiQAK,Evelyn,Y,Shinsato,7509 CIRCLE HILL DR,NaN,OAKLAND,CA,94605,ALAMEDA,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
180,00341000019SYVcAAO,0014100000x9FJUAA2,Jon,NaN,Haghayeghi,372 OAKDALE DR,NaN,CLAREMONT,CA,91711,LOS ANGELES,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
198,0034100001ABK9FAAX,0014100000yG9vxAAC,Beth,NaN,Jackson,841 SWEETSER AVE,APT 2,NOVATO,CA,94945,MARIN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
264,00341000019eaoGAAQ,0014100000xOsktAAC,Rena,NaN,Witter,20311 SHERMAN WAY,APT 105,WINNETKA,CA,91306,LOS ANGELES,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN


##### Save the to the analytics schema in civis

In [19]:
civis.io.dataframe_to_civis(contacts_demographics, database = db, table = 'ibrahima.contacts_demographics',
                             existing_table_rows='drop')

<CivisFuture at 0x1628a6ef0 state=running>